# Keras Tuner Bars

In [1]:
import h5py
import numpy as np
import tensorflow
from tensorflow import keras

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)

tensorflow 2.1.0
keras 2.2.4-tf


In [2]:
class HDF5DataGeneratorBars(keras.utils.Sequence):
    def __init__(self, file, num_classes, num_dp, batch_size=100, num_batches=None):
        self.file = h5py.File(file, "r")
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.num_dp = num_dp

        if num_batches == None:
            self.num_batches = int(
                np.floor(self.file["multiplicity"].len() / self.batch_size)
            )
        else:
            self.num_batches = num_batches

    def __len__(self):
        return self.num_batches

    def __getitem__(self, index):
        x, y = self.getbatch(index)
        np.place(y, y >= self.num_classes, self.num_classes - 1)  # FIXME
        y = keras.utils.to_categorical(y, num_classes=self.num_classes)
        return x, y

    def getbatch(self, index):
        "Generate one batch of data"
        start = self.batch_size * index
        end = self.batch_size * (index + 1)
        x = np.array(self.file["flatfeatures"][start:end])
        # multiplicity column 2 -> label = 'nPH'
        y = np.array(self.file["multiplicity"][start:end, [2]].ravel())
        return x, y

In [3]:
class Model():
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self):
        model = keras.models.Sequential()
        model.add(keras.layers.Flatten()
        model.add(keras.layers.Dense(units=9000, activation="relu"))
        model.add(keras.layers.Dense(units=1200, activation="relu"))
        model.add(keras.layers.Dense(units=self.num_classes, activation="softmax"))

        optimizer = keras.optimizers.Adagrad()#lr=0.01, epsilon=None, decay=0.0)
        # optimizer = keras.optimizers.Adam()
        loss = keras.losses.CategoricalCrossentropy()

        model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
        return model

In [4]:
num_dp = 30
max_neutrons = 5

file = f"data/600AMeV_{num_dp}dp.bars-shuffled.h5"
generator = HDF5DataGeneratorBars(file, num_classes=max_neutrons + 1, num_dp=num_dp)
model = Model(max_neutrons + 1).build()

In [5]:
history = model.fit(generator, epochs=10)
model.save("03c-keras-bars-V3.h5")

Train for 50000 steps
Epoch 1/10
28079/50000 [===============>..............] - ETA: 32:52 - loss: 47351.4791 - accuracy: 0.6127

KeyboardInterrupt: 

In [ ]:
tuner.results_summary()